# Knowledge and Data: Practical Assignment 1 
## Manipulate Propositional Logic and Simple Knowledge Graph Logic

- YOUR NAME: Francesca Urgese

- YOUR VUNetID: 2736482

*(If you do not provide your NAME and VUNetID we will not accept your submission.)*

### Learning objectives

At the end of this exercise you should be able to perform some simple manipulations of two different formal systems: 
1. You will be able to transform Propositional Logic statements from one syntactic form to semantically equivalent  alternative representation.
2. You will be able to evaluate a Propositional Logic statements given some validation of the propositional variables. 
3. You will be able to transform one Knowledge Graph into an equivalent one with a different Syntax 
4. You will be able to calculate simple entailment between two Knowledge Graph

### Practicalities

Follow this Notebook step-by-step. Some of the exercises are already 
fully filled in to give you some examples of how to do things. Others are
partially filled, and require you to fill in some gaps. And others, you 
will have to fully program yourself. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everythink is filled in and works, save the Notebook and submit it 
as a Jupyter Notebook (i.e. with an ipynb extension). **Please use as name of the 
Notebook your studentID+Assignment1.ipynb**.  

Other than in courses dedicated to programming we will not evaluate the style
of the programs. But we will test your programs on other data than we provide, 
and your program should give the correct answers to those test-data as well. 

## Exercises related to Propositional Logic 

We will use a very simple, though crude and ugly, representation for Propositional Logic: Propositions are strings, and formulas are lists, starting with an operator as first element in the list, and the subformulas as second and (if appropriate) third argument. (Note that this is slightly different than the definition in the lecture, where Propositions are lists with one argument). These are formulas in Prefix Notation.

In the file, there is one formula per line. 

For simplicity, we will only consider formulas with any of the three variables P, Q and R, and binary operators ">> (implies), & (and), | (or)" and "~ (not)". 

Here are some formulas as examples.

(~R & P) >> Q

~(((P | ~Q) >> R) >> (P & R)) 

~(P >> Q) | (R >> P) 

(P >> Q) | (Q >> ~P)


The first thing you will have to do is to open the file. The code below will read every line of the file into a list called *content*.

Do *not* forget to run the following cell, otherwise, the file will not be opened. import os 


In [3]:
import os 
from logic import *

In [4]:
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, 'PLknowledgebase.txt')
print(filename)
with open(filename) as file:
    content = file.readlines()
     
print(content)

C:\Users\francesca\Documents\code\100-Days-Of-Coding\Assignment_1\PLknowledgebase.txt
['(~R & P) >> Q\n', '~(((P | ~Q) >> R) >> (P & R)) \n', '~(P >> Q) | (R >> P) \n', '(P >> Q) | (Q >> ~P)']


In [5]:
P, Q, R = vars('P', 'Q', 'R')

for line in content:
    formula = eval(line.strip())
    print()
    print(formula)
    print(formula.is_tautology())
    print(type(formula))
    print(formula.children[0])


(¬R ^ P) => Q
False
<class 'logic.Implies'>
¬R ^ P

¬(((P v ¬Q) => R) => (P ^ R))
False
<class 'logic.Not'>
((P v ¬Q) => R) => (P ^ R)

¬(P => Q) v (R => P)
False
<class 'logic.Or'>
¬(P => Q)

(P => Q) v (Q => ¬P)
True
<class 'logic.Or'>
P => Q


The last procedure makes use of the inbuilt properties of the logic package. Let us do similar stuff ourselves. 

### Task 1: (1 Point) Complete the procedure *prefix* that transforms the formula into Prefix notation, i.e., the operator "sits" in front of the two arguments.
Also use different symbols for operators, IMP for implication, AND for conjunction, OR for disjunction and NEG for negation. 

In [6]:
def prefix(s):
    if type(s) is Or:
        child1 = s.children[0]
        child2 = s.children[1]
        string = "OR(" + prefix(child1) + "," + prefix(child2) + ")"
        return string
    ## Fill in the remaining cases (hint: not every formula has the same number of children) 
    elif type(s) is And:
        child1 = s.children[0]
        child2 = s.children[1]
        string = "AND(" + prefix(child1) + "," + prefix(child2) + ")"
        return string
    elif type(s) is Implies:
        child1 = s.children[0]
        child2 = s.children[1]
        string = "IMP(" + prefix(child1) + "," + prefix(child2) + ")"
        return string
    elif type(s) is Not: 
        child1 = s.children[0]
        string = "NEG(" + prefix(child1) + ")"
        return string 
    else:
        return str(s)

Execute the following line of code to check whether your procedure works correctly or not.

In [7]:
for line in content:
    formula = eval(line.strip())
    spacing = 30 - len(str(formula))
    print(str(formula) + " "*spacing + "== " + prefix(formula))

(¬R ^ P) => Q                 == IMP(AND(NEG(R),P),Q)
¬(((P v ¬Q) => R) => (P ^ R)) == NEG(IMP(IMP(OR(P,NEG(Q)),R),AND(P,R)))
¬(P => Q) v (R => P)          == OR(NEG(IMP(P,Q)),IMP(R,P))
(P => Q) v (Q => ¬P)          == OR(IMP(P,Q),IMP(Q,NEG(P)))


If you run your procedure on the input data, you should get something like the following output: 

IMP(AND(NEG(R),P),Q) 

etc

### Task 2: (2 Points) Write a procedure *evaluate* that calculates the value of a formula. 

Input to the procedure is the formula s, and the three truth values for variable P, Q and R. 

This is a procedural way to implement the "meaning" of the operators (which you can read from the Truth Tables)

In [8]:
def evaluate(s, p, q, r):
    if type(s) is Or:
        child1 = s.children[0]
        child2 = s.children[1]
        if (evaluate(child1,p,q,r) is True) and (evaluate(child2,p,q,r) is True):
            return True
        if (evaluate(child1,p,q,r) is True) and (evaluate(child2,p,q,r) is False):
            return True
        if (evaluate(child1,p,q,r) is False) and (evaluate(child2,p,q,r) is True):
            return True
        if (evaluate(child1,p,q,r) is False) and (evaluate(child2,p,q,r) is False):
            return False
    
    elif type(s) is Implies:
        child1 = s.children[0]
        child2 = s.children[1]
        if (evaluate(child1,p,q,r) is False) and (evaluate(child2,p,q,r) is True):
            return True  
        if (evaluate(child1,p,q,r) is False) and (evaluate(child2,p,q,r) is False): 
            return True
        if (evaluate(child1,p,q,r) is True) and (evaluate(child2,p,q,r) is True):
            return True
        if (evaluate(child1,p,q,r) is True) and (evaluate(child2,p,q,r) is False):
            return False
    
    elif type(s) is And:
        child1 = s.children[0]
        child2 = s.children[1]
        if (evaluate(child1,p,q,r) is True) and (evaluate(child2,p,q,r) is True):
            return True
        if (evaluate(child1,p,q,r) is True) and (evaluate(child2,p,q,r) is False):
            return False
        if (evaluate(child1,p,q,r) is False) and (evaluate(child2,p,q,r) is True):
            return False
        if (evaluate(child1,p,q,r) is False) and (evaluate(child2,p,q,r) is False):
            return False
    
    elif type(s) is Not:
        child1 = s.children[0]
        if (evaluate(child1,p,q,r) is True):
            return False
        if (evaluate(child1,p,q,r) is False):
            return True
    
    
    elif type(s) is Variable:        
        if str(s) == 'P':
            return p
        elif str(s) == 'Q':
            return q
        elif str(s) == 'R':
            return r
    else:
        print("Something went wrong") 

Execute the following line of code to check whether your procedure works correctly or not.

In [9]:
for l in content:
    formula = l.strip()
    spacing = 30 - len(str(formula))
    print(str(formula) + " "*spacing + str(evaluate(s=eval(formula), p=True, q=True, r=False)))

(~R & P) >> Q                 True
~(((P | ~Q) >> R) >> (P & R)) False
~(P >> Q) | (R >> P)          True
(P >> Q) | (Q >> ~P)          True


If you run your program on the given input, you should get: <br />
True<br />
False<br />
True<br />
True<br />

### Task 3: (1 Points) Write a procedure *tautology* that calculates whether a given formula (with maximally three variables P,Q and R) is a tautology 
Hint: Use the *evaluate* function you just wrote and implement a "truthtable" approach whether the formula is always evaluated as "true". Do *not* use the 'is_tautology' method seen earlier.

In [39]:
def tautology(s):
    if evaluate(s=eval(formula), p=True, q=True, r=True) == False or evaluate(s=eval(formula), p=True, q=True, r=False) == False or  evaluate(s=eval(formula), p=True, q=False, r=True) == False or  evaluate(s=eval(formula), p=True, q=False, r=False) == False or  evaluate(s=eval(formula), p=False, q=True, r=True) == False or  evaluate(s=eval(formula), p=False, q=True, r=False) == False or  evaluate(s=eval(formula), p=False, q=False, r=True) == False or  evaluate(s=eval(formula), p=False, q=False, r=False) == False:
        return False 
    else: 
        return True
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    


Execute the following line of code to check whether your procedure works correctly or not.

In [40]:
print("Formula" + 23*" " + "Tautology")
for l in content:
    formula = l.strip()
    spacing = 30 - len(str(formula))

    print(str(formula) + " "*spacing + (repr(tautology(formula))))

Formula                       Tautology
(~R & P) >> Q                 False
~(((P | ~Q) >> R) >> (P & R)) False
~(P >> Q) | (R >> P)          False
(P >> Q) | (Q >> ~P)          True


When you evaluate your code on the provided input, you should get: <br />
False<br />
False<br />
False<br />
True<br />

## Exercises related to Simple Knowledge Graph Logic 

We will use a very simple, though crude and ugly, representation for Simple Knowledge Graph Logic: Resources are strings, and triples are lists of Resources. So, a triple ( s p o ) will be represented as a list [s,p,o], and a Knowledge Graph with two triples (s1, p1, o1) and (s2, p2, o2) as a list containing two lists: [[s1,p1,o1], [s2,p2,o2]].

Every line contains a knowledge graph (one list). 

Run the following code to load the Knowledge Graph in knowledgegraph.txt

In [ ]:
import os 
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, 'knowledgegraph.txt')
with open(filename) as file:
    content = [[t for t in eval(g)] for g in file.readlines()]
print(content)

[[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']], [['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'type', 'Capital']], [['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'type', 'Capital'], ['Capital', 'type', 'City'], ['Netherlands', 'neighbours', 'Belgium'], ['Netherlands', 'type', 'Country']]]


Maybe you want to write a function to print this graph for convenience ... (no extra points, though) 

The next two exercises are about syntactic transformation of Knowledge Graphs. The two target formats are simplified versions of syntaxes for the language RDF that we will introduce later. 

### Task 4a: (1 Point) Write a procedure to transform the Knowledge Graphs into N-triple syntax. 
N-triple is a simple (the most simple?) representation format for representing Knowledge Graphs. 

A triple (s,p,o) is simply written as as follows:

     <s> <p> <o> . 
     
As one string, with blanks between s,p and o, with < and > around the variables, and a dot after the triple. 

First, write a function that transforms a triple ['s','p','o'] into a single string "\<s> \<p> \<o> ."

In [ ]:
def as_ntriple(triple):
    return "<{}> <{}> <{}>.".format(*triple)

In [ ]:
def as_ntriples(graph):
    return([as_ntriple(l) for l in graph]) 


Execute the following line of code to check whether your procedure works correctly or not.

In [ ]:
for l in content:
     print(as_ntriples(l))

['<s> <p> <o>.', '<s> <p1> <o2>.', '<s2> <p2> <o>.']
['<Netherlands> <name> <"Netherlands">.', '<Netherlands> <has_capital> <Amsterdam>.', '<Amsterdam> <type> <Capital>.']
['<Netherlands> <name> <"Netherlands">.', '<Netherlands> <has_capital> <Amsterdam>.', '<Amsterdam> <type> <Capital>.', '<Capital> <type> <City>.', '<Netherlands> <neighbours> <Belgium>.', '<Netherlands> <type> <Country>.']


Your answer should look like this: <br />
['\<s> \<p> \<o> .', '\<s> \<p1> \<o2> .', '\<s2> \<p2> \<o> .']<br/>
['\<Netherlands> \<name> \<"Netherlands"> .', '\<Netherlands> \<has_capital> \<Amsterdam> .', '\<Amsterdam> \<type> \<Capital> .']<br/>
['\<Netherlands> \<name> \<"Netherlands"> .', '\<Netherlands> \<has_capital> \<Amsterdam> .', '\<Amsterdam> \<type> \<Capital> .', '\<Capital> \<type> \<City> .', '\<Netherlands> \<neighbours> \<Belgium> .', '\<Netherlands> \<type> \<Country> .']

### Task 4b (1 Point) Adjust your procedure to cope with literal objects


In reality, the N-triples format specifies that only objects must be embedded between angled brackets; literal values, such as strings and numbers, are to be represented as string values, for example "\<s> \<p> \"12\" .". Since knowledge graphs are construced with an object-centric language (RDF), we only have to consider the last element of a triple.

Adjust the function _as\_triple_ to make it output valid N-triples.

In [ ]:
def as_ntriple(triple):
    for i in triple:
        if i == "Netherlands": 
            return "<{}> <{}> {}.".format(*triple)
        else:
            return "<{}> <{}> <{}>.".format(*triple)    
            
    
            

Execute the following line of code to check whether your procedure works correctly or not.

In [ ]:
for l in content:
     print(as_ntriples(l))

['<s> <p> <o>.', '<s> <p1> <o2>.', '<s2> <p2> <o>.']
['<Netherlands> <name> "Netherlands".', '<Netherlands> <has_capital> Amsterdam.', '<Amsterdam> <type> <Capital>.']
['<Netherlands> <name> "Netherlands".', '<Netherlands> <has_capital> Amsterdam.', '<Amsterdam> <type> <Capital>.', '<Capital> <type> <City>.', '<Netherlands> <neighbours> Belgium.', '<Netherlands> <type> Country.']


Your answer should look like this: <br />
['\<s> \<p> \<o> .', '\<s> \<p1> \<o2> .', '\<s2> \<p2> \<o> .']<br/>
['\<Netherlands> \<name> "Netherlands" .', '\<Netherlands> \<has_capital> \<Amsterdam> .', '\<Amsterdam> \<type> \<Capital> .']<br/>
['\<Netherlands> \<name> "Netherlands" .', '\<Netherlands> \<has_capital> \<Amsterdam> .', '\<Amsterdam> \<type> \<Capital> .', '\<Capital> \<type> \<City> .', '\<Netherlands> \<neighbours> \<Belgium> .', '\<Netherlands> \<type> \<Country> .']

### Task 5: (2 Points) Write a procedure to evaluate whether a Triple (s p o) is entailed by your Knowledge Graph

Remember that we can calculate whether a triple is entailed by a Knowledge Graph (w.r.t Simple Knowledge Graph Logic) by simply checking whether the triple is a subgraph of the knowledge graph.  

The triples, on which you can test your implementations are located in the file: *entailedTriples.txt*. You need to open the file and check for all triples whether it is entailed by the knowledge graphs in knowledgegraph.txt (which is already opened). Hint: you can use *eval* to convert a raw string to a list.

You might want to add auxiliary functions in extra cells if necessary.

In [ ]:
def tripleEntail(i, l):   
    if i in l:
        return "{} entails {}".format(l, i)
    else:
        return "{} does NOT entail {}".format(l, i)
    

Provide some code to check that your function works correctly.  

In [ ]:
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, 'entailedTriples.txt')
with open(filename) as file:
    EntailedTriples = [[t for t in eval(g)] for g in file.readlines()] 
    
for l in content:
    for i in EntailedTriples:
        print(tripleEntail(i, l))
    

[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] entails ['s2', 'p2', 'o']
[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] does NOT entail ['Netherlands', 'name', '"Netherlands"']
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'type', 'Capital']] does NOT entail ['s2', 'p2', 'o']
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'type', 'Capital']] entails ['Netherlands', 'name', '"Netherlands"']
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'type', 'Capital'], ['Capital', 'type', 'City'], ['Netherlands', 'neighbours', 'Belgium'], ['Netherlands', 'type', 'Country']] does NOT entail ['s2', 'p2', 'o']
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'type', 'Capital'], ['Capital', 'type', 'City'], ['Netherlands', 'neighbours', 'Belgium'], ['Netherlands', 'type'

The expected output is a list of relations between graphs and triples<br />
[['s','p','o'],['s','p1','o2'],['s2','p2','o']] entails ['s2','p2','o']<br />
[['s','p','o'],['s','p1','o2'],['s2','p2','o']] does NOT entail ['Netherlands','name','"Netherlands"']<br />
and so on for all graphs and triples. 

### Task 6: (2 Points) Write a procedure to evaluate whether one Knowledge Graph is entailed by another Knowledge Graph

The graphs, on which you can test your implementations are located in the file: entailedGraphs.txt. 

You might want to add auxiliary functions in extra cells if necessary.

In [ ]:
def graphEntail(i, l):

    for b in range(len(i)):
        if i[b] in l:
            return "{} entails {}".format(l, i)
        else:
            return "{} does NOT entail {}".format(l, i)
        
    

    
    
    

Provide some code to check that your function works correctly.  

In [ ]:
fileDir = os.path.dirname(os.path.realpath('__file__'))
filename = os.path.join(fileDir, 'entailedGraphs.txt')
with open(filename) as file:
    EntailedGraphs = [[t for t in eval(g)] for g in file.readlines()]
    
for l in content:
    for i in EntailedGraphs:
        print(graphEntail(i, l))
    

[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] entails [['s2', 'p2', 'o'], ['s2', 'p2', 'o']]
[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] entails [['s', 'p', 'o'], ['s2', 'p2', 'o']]
[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] does NOT entail [['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Capital', 'a', 'City'], ['Netherlands', 'neighbours', 'Belgium']]
[['s', 'p', 'o'], ['s', 'p1', 'o2'], ['s2', 'p2', 'o']] does NOT entail [['Netherlands', 'has_capital', 'Amsterdam'], ['Capital', 'a', 'Country'], ['Netherlands', 'neighbours', 'Belgium']]
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'type', 'Capital']] does NOT entail [['s2', 'p2', 'o'], ['s2', 'p2', 'o']]
[['Netherlands', 'name', '"Netherlands"'], ['Netherlands', 'has_capital', 'Amsterdam'], ['Amsterdam', 'type', 'Capital']] does NOT entail [['s', 'p', 'o'], ['s2', 'p2', 'o']]
[['Netherlands', 'name', '"

The expected output is a list of relations between graphs and graphs<br />
[['s','p','o'],['s','p1','o2'],['s2','p2','o']] entails [['s','p','o'],['s2','p2','o']]<br />
[['s','p','o'],['s','p1','o2'],['s2','p2','o']] entails [['s2','p2','o'],['s2','p2','o']]<br />
[['s','p','o'],['s','p1','o2'],['s2','p2','o']] does NOT entail [['Netherlands','name','"Netherlands"'],['Netherlands','has_capital','Amsterdam'],['Capital','a','City'],['Netherlands','neighbours','Belgium']]
 [['s','p','o'],['s2','p2','o']]<br />
and so on for all graphs and triples. 